<a href="https://colab.research.google.com/github/esraa-abdelmaksoud/Shai-Training-Notebooks/blob/main/Task_6_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [3]:
# CODE HERE
from sklearn.linear_model import LinearRegression

linear = LinearRegression()
linear.fit(housing_prepared, housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [4]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [12]:
# CODE HERE
some_data_tr = full_pipeline.transform(some_data)
pred = linear.predict(some_data_tr)
print(pred)
print(np.array(some_labels))

[181746.54359616 290558.74973505 244957.50017771 146498.51061398
 163230.42393939]
[103000. 382100. 172600.  93400.  96500.]


# measure this regression model’s RMSE on the whole training set 
* sing Scikit-Learn’s mean_squared_error() function:

In [13]:
from sklearn.metrics import mean_squared_error

In [15]:
# CODE HERE
housing_preds = linear.predict(housing_prepared)
linear_rmse = mean_squared_error(housing_labels, housing_preds, squared=False)
linear_rmse


67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

The root mean squred error is 67593. It is high which means another model with a different algorithm should be trained.

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [16]:
from sklearn.tree import DecisionTreeRegressor 

In [17]:
# CODE HERE
tree = DecisionTreeRegressor()
tree.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [18]:
# CODE HERE
housing_tree_preds = tree.predict(housing_prepared)
tree_rmse = mean_squared_error(housing_labels, housing_tree_preds, squared=False)
tree_rmse

0.0

# Explaine this result 
write down your answar

The rmse of decision tree is 0 which means the model has an overfitting problem.

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [19]:
from sklearn.model_selection import cross_val_score

In [23]:
# CODE HERE
score = cross_val_score(tree, housing_prepared, housing_labels, cv=10,
                        scoring="neg_mean_squared_error")
tree_score = np.sqrt(-score)
tree_score

array([65923.09013902, 70524.52667343, 69988.39750289, 70668.86865182,
       72158.10789585, 66820.30011888, 66091.51009865, 68937.244088  ,
       64506.87514742, 70130.88687103])

2- display the resultant scores and calculate its Mean and Standard deviation

In [25]:
# CODE HERE
print("Scores:", tree_score, "\n Mean:", tree_score.mean(), "\n Std:",
      tree_score.std())

Scores: [65923.09013902 70524.52667343 69988.39750289 70668.86865182
 72158.10789585 66820.30011888 66091.51009865 68937.244088
 64506.87514742 70130.88687103] 
 Mean: 68574.98071870054 
 Std: 2416.655935312109


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [ ]:
# CODE HERE

## Let’s train one last model the RandomForestRegressor.

In [ ]:
# CODE HERE

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [ ]:
# CODE HERE

# Save every model you experiment with 
*using the joblib library*

In [ ]:
# CODE HERE

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# CODE HERE

with the evaluation scores

In [ ]:
# CODE HERE

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [ ]:
# CODE HERE

2-display these importance scores next to their corresponding attribute names:

In [ ]:
# CODE HERE

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [ ]:
# CODE HERE

2-run your full_pipeline to transform the data

In [ ]:
# CODE HERE

3-evaluate the final model on the test set

In [ ]:
# CODE HERE

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [ ]:
from scipy import stats

In [ ]:
# CODE HERE

# Great Job!
# #shAI_Club